# Machine Intelligence II - Team MensaNord
## Sheet 06

- Nikolai Zaki
- Alexander Moore
- Johannes Rieke
- Georg Hoelger
- Oliver Atanaszov

In [1]:
from __future__ import division, print_function
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import scipy.io.wavfile
import scipy.stats
import numpy as np

## Exercise 1
### A

In [39]:
def E_s(W, s):
    W_local = np.copy(W)
    for col in range(W_local.shape[0]):
        W_local[col, :] *= s[col]
        W_local[:, col] *= s[col]
    # print("# ", W_local)
    # print("# s:", s, "\tE:", - np.sum(W_local) / 2)
    return - np.sum(W_local) / 2

# Test
A = np.array([[1, 2, 3], [1, 2, 3], [1, 2, 3]])
B = np.array([1, 0, 2])
for i in range(2):
    print(A)
    print(E_s(A, B))
    print()

[[1 2 3]
 [1 2 3]
 [1 2 3]]
-10.5

[[1 2 3]
 [1 2 3]
 [1 2 3]]
-10.5



In [67]:
# inintialization

# loop borders
t_max = 1000
M = 1

# Temperature ( beta == noise ) decreasing with factor tau
beta = np.random.random() / 50
print(1 / beta) # == Temperature

tau = np.random.random() + 2
print(tau)

# configuration s
s = np.random.choice([-1, 1], 6)
print(s)

# weight matrix, symmetrical, diagonal 0
W = np.random.random(size=(6, 6))
W = (W + W.T)/2
for i in range(W.shape[0]):
    W[i, i] = 0
print(W)

159.73055198128083
2.967608207023197
[-1  1 -1 -1 -1  1]
[[ 0.          0.41575661  0.44753002  0.61907623  0.42404272  0.67292749]
 [ 0.41575661  0.          0.60592311  0.18712279  0.58502223  0.41133932]
 [ 0.44753002  0.60592311  0.          0.32025882  0.29969132  0.50733595]
 [ 0.61907623  0.18712279  0.32025882  0.          0.23174447  0.62394721]
 [ 0.42404272  0.58502223  0.29969132  0.23174447  0.          0.64434177]
 [ 0.67292749  0.41133932  0.50733595  0.62394721  0.64434177  0.        ]]


In [70]:
validation_min = E_s(W, s) # for validation
print("start min", validation_min)

for t in range(t_max):
    for m in range(M):        
        i = np.random.randint(0, 6)
        s_local = np.copy(s)
        s_local[i] *= -1
        E_1 = E_s(W, s)
        E_2 = E_s(W, s_local)
        E_d = E_2 - E_1
        P = 1 / (1 + np.exp(beta*E_d))
        # print("\nt:", t, " i:", i, "\n s1:", s, "\tE1:", E_1, "\n s2:", s_local, "\tE2:", E_2)
        if np.random.random() < P:
            s = np.copy(s_local)
            # print("new s")
        if E_s(W, s) < validation_min:
            validation_min = E_s(W, s)
            # print("new min")
    beta *= tau
print("s optimal", s)
print("E optimal", E_s(W, s))
print("min for reference", validation_min)

start min -6.9960600629
s optimal [-1 -1 -1 -1 -1 -1]
E optimal -6.9960600629
min for reference -6.9960600629
